In [12]:
# Some steps required until these Python modules are properly installed...
import sys
sys.path.append('../../../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
import gbr
import fiona
%pylab inline
import pandas as pd
import geopandas as gpd
import numpy as py
# display more then one result below cell
from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


In [13]:
# makes the tables a bit more readable
CSS = """
body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}
"""
from IPython.core.display import HTML
HTML('<style>{}</style>'.format(CSS))

In [14]:
# Read in the Current load and all change paramters files and create new columns to track version
cl = pd.read_csv("d:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/cc/ParameterTable.csv", error_bad_lines=False)
ac = pd.read_csv("d:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/all_change/ParameterTable.csv", error_bad_lines=False)
cl['version'] = "cl"
ac['version'] = "ac"
# Display
display (cl [0:1])
display (len (cl))
display (ac [0:1])
display (len (ac))

b'Skipping line 63642: expected 7 fields, saw 8\n'
b'Skipping line 231401: expected 7 fields, saw 8\n'
b'Skipping line 1475880: expected 7 fields, saw 8\nSkipping line 1476799: expected 7 fields, saw 8\nSkipping line 1477718: expected 7 fields, saw 8\n'


,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
0,SC #1608,link for catchment SC #1608,Water,Sediment - Fine,Sediment Generation (EMC & Gully) - SedNet,fineEMCmodel,RiverSystem.Catchments.Models.ContaminantGener...,cl


1477775

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
0,SC #1608,link for catchment SC #1608,Water,Sediment - Fine,Sediment Generation (EMC & Gully) - SedNet,fineEMCmodel,RiverSystem.Catchments.Models.ContaminantGener...,ac


1477770

In [15]:
#Join all the data together and ignore indexes so it all gets added
full_set = pd.concat([cl,ac],ignore_index=True)
display (full_set [0:1])
display (len (full_set))

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
0,SC #1608,link for catchment SC #1608,Water,Sediment - Fine,Sediment Generation (EMC & Gully) - SedNet,fineEMCmodel,RiverSystem.Catchments.Models.ContaminantGener...,cl


2955545

In [16]:
# Let's see what changes in the main columns we care about
changes = full_set.drop_duplicates(subset=['CATCHMENT','LINK','ELEMENT','CONSTITUENT','MODEL','PARAMETER','VALUE'], keep=False)
display (changes [0:1])
display (len (changes))

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
5139,SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.908405731,cl


3497

In [17]:
changes.sort_values(['CATCHMENT'], ascending=[True])
changes [0:2]

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
5139,SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.908405731,cl
8579,SC #655,link for catchment SC #655,link for catchment SC #655,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.957812755,cl


In [18]:
#Pull out the old and new data into separate dataframes
change_cl = changes[(changes["version"] == "cl")]
change_ac = changes[(changes["version"] == "ac")]
display (change_cl [0:1])
display (len (change_cl))
display (change_ac [0:1])
display (len (change_ac))

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
5139,SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.908405731,cl


1751

,CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,VALUE,version
1482914,SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.907279812,ac


1746

In [19]:
#Index on the account numbers
change_cl.set_index(['CATCHMENT','LINK','ELEMENT','CONSTITUENT','MODEL','PARAMETER'],inplace=True)
change_ac.set_index(['CATCHMENT','LINK','ELEMENT','CONSTITUENT','MODEL','PARAMETER'],inplace=True)
display (change_cl [0:1])
display (len (change_cl))
display (change_ac [0:1])
display (len (change_ac))

,,,,,,VALUE,version
CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,,
SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.908405731,cl


1751

,,,,,,VALUE,version
CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,,
SC #1658,link for catchment SC #1658,link for catchment SC #1658,Sediment - Fine,In Stream Fine Sediment Model - SedNet,Bank Erosion Management Factor,0.907279812,ac


1746

In [20]:
df = pd.concat([change_cl, change_ac], axis=1)
display (df [0:1])
display (len (df))

,,,,,,VALUE,version,VALUE,version
CATCHMENT,LINK,ELEMENT,CONSTITUENT,MODEL,PARAMETER,,,,
BFD,BFD,BFD,NaN,Storage,Evaporation Volume,Water volume 0;,cl,Water volume 611784.016728089;,ac


1751

In [22]:
df.to_excel('D:/a_sc_p2r_b_rc7/Model_runs/BD_1/Runs/a_scen_reader/RC7/Parameter_changes_cl_ac.xlsx')